<a href="https://colab.research.google.com/github/gorkemozkaya/nmt-en-tr/blob/master/Turkish_English_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running the pre-trained NMT models in Python

This notebook illustrates how one can load the pre-trained models and run it on new Turkish or English sentences for translation. It is mostly based on [this](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb) tensor2tensor notebook from the official [tensor2tensor](https://github.com/tensorflow/tensor2tensor) repository. 

In [0]:
!pip install --quiet tensorflow==1.14 tensor2tensor==1.13.4

In [0]:
import os

# Enable TF Eager execution
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
tfe = tf.contrib.eager
tfe.enable_eager_execution() 

import numpy as np
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.data_generators import text_encoder

from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry

## Clone the repository and import the module

In [3]:
!git clone https://github.com/gorkemozkaya/nmt-en-tr.git||true

fatal: destination path 'nmt-en-tr' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append("nmt-en-tr")
import nmt_en_tr

## Downloading pretrained model

In [0]:
!wget -nc -q https://github.com/gorkemozkaya/nmt-en-tr/releases/download/pretrained_model/en2tr.zip
!unzip -n -qq en2tr.zip

In [0]:
model_path = 'en2tr'

data_dir = os.path.join(model_path, 'data')
ckpt_dir = os.path.join(model_path, 'model')

en2tr_problem = problems.problem("translate_en_tr")
encoders = en2tr_problem.feature_encoders(data_dir)

ckpt_path = tf.train.latest_checkpoint(ckpt_dir)



In [0]:
# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id 
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers)) 

In [0]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_tpu"

# Other setup
Modes = tf.estimator.ModeKeys

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_en_tr")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

In [9]:
# Restore and translate!
def translate(inputs, **kwargs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path):
    model_output = translate_model.infer(encoded_inputs, **kwargs)["outputs"]
  if len(model_output.shape) == 2:
    return decode(model_output)
  else:
    return [decode(x) for x in model_output[0]]

inputs = "If Turkey provides a competitive, safe, and predictable business and \
investment environment, it can reach high growth rates and development levels, \
with its alternative tourism opportunities, agriculture, young, educated \
population, and entrepreneurial spirit."

outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

Inputs: If Turkey provides a competitive, safe, and predictable business and investment environment, it can reach high growth rates and development levels, with its alternative tourism opportunities, agriculture, young, educated population, and entrepreneurial spirit.
Outputs: Türkiye rekabetçi, güvenli ve öngörülebilir bir iş ve yatırım ortamı sağladığı takdirde, alternatif turizm fırsatları, tarım, genç, eğitimli genç ve girişimcilik ruhuyla yüksek büyüme oranları ve kalkınma seviyelerine ulaşabilir.
